# **How to use**

- First, run **Neccessary setup** and **Define essential functions** cells.
- Then, record your query using **Record your query here** cell.
- After having an audio file, either run **English Virtual Assistant** or **Vietnamese Virtual Assistant** cell according to your language. 

__Note__: answers are only taken from wikipedia

# **Neccessary setup**

In [1]:
!pip install ffmpeg-python bs4 google gtts pygobject pydub jiwer SpeechRecognition -q

     |████████████████████████████████| 32.8 MB 1.4 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
import glob
import os
import io
import requests
import librosa
import matplotlib.pyplot as plt
import ffmpeg
import scipy
import scipy.io.wavfile
import speech_recognition as sr
import IPython

from tensorflow import keras
from tensorflow.keras import layers
from jiwer import wer
from bs4 import BeautifulSoup
from gtts import gTTS
#from IPython import display

from IPython.display import HTML, Audio
from pydub import AudioSegment
from google.colab.output import eval_js
from base64 import b64decode

try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")

In [3]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = scipy.io.wavfile.read(io.BytesIO(riff))

  return audio, sr

In [4]:
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [5]:
!gdown --id 118u3TxwFr82Q2NUHwMonExVf385A84EM

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=118u3TxwFr82Q2NUHwMonExVf385A84EM
To: /content/model.zip
100% 296M/296M [00:01<00:00, 185MB/s]


In [6]:
!unzip model.zip

Archive:  model.zip
   creating: model/
  inflating: model/keras_metadata.pb  
   creating: model/assets/
  inflating: model/saved_model.pb    
   creating: model/variables/
  inflating: model/variables/variables.data-00000-of-00001  
  inflating: model/variables/variables.index  


# **Define essential functions**

In [7]:
def google_search(query, lang="en", num=10):
  results = list(search(query, tld="co.in", lang='en', num=num, stop=num, pause=2))
  
  results = [link for link in results if f'{lang}.wikipedia' in link] # just consider only wikipedia webpages

  return results

def extract_p_content(url, length_content=5):
  # Fetch URL Content
  r = requests.get(url)
  
  # Get body content
  soup = BeautifulSoup(r.text,'html.parser').select('body')[0]

  # Initialize variable
  paragraphs = []
  
  # Iterate through all tags
  for tag in soup.find_all():
      
      # For Paragraph use p tag
      if tag.name=="p":
        
          # use text for fetch the content inside p tag
          paragraphs.append(tag.text)
          
  page_content = "".join(paragraphs[1:length_content])

  return page_content

def text_to_speech(content, lang="en", output_filename="output.mp3"):
  output = gTTS(content, lang=lang, slow=False) # text to speech

  output.save(output_filename) # save google audio to a file

  data, sr = librosa.load(output_filename) # load google audio using librosa library
  IPython.display.display(IPython.display.Audio(np.transpose(data), rate=sr)) # display audio 

## For using trained model from tensorflow

In [8]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

def encode_single_sample(wav_file, frame_length=256, frame_step=160, fft_length=384):
    # Downsampling the file
    audio, _ = librosa.load(wav_file, sr=16000)
    # Change type to float
    audio = tf.cast(audio, tf.float32)
    # Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    # We only need the magnitude, which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    # normalisation
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)

    return spectrogram

# The set of characters accepted in the transcription.
lowercase_chars = string.ascii_lowercase
accented_chars = "àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹý"
final_chars = lowercase_chars + accented_chars  + " "
characters = [x for x in final_chars]
# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

In [9]:
model_path = "./model"
model = tf.keras.models.load_model(model_path, custom_objects={'CTCLoss': CTCLoss})

# **Record your query here**

In [10]:
audio_filename = "record.wav" # file .wav
audio, sampling_rate = get_audio() # recording, please enable colab to use your microphone
scipy.io.wavfile.write(audio_filename, sampling_rate, audio) # save your recorded audio to .wav file

# **English Virtual Assistant**

## Read audio file and perform ASR

In [11]:
r = sr.Recognizer()
my_audio = sr.AudioFile(audio_filename)
with my_audio as source:
  audio = r.record(source)

your_speech = r.recognize_google(audio)
print("Audio transcription: ", your_speech)

Audio transcription:  who is Abraham Lincoln


## Perform google search and get the first wikipedia webpage

In [12]:
google_lang = "en"

query = your_speech
retrieved_webpages = google_search(query, lang=google_lang)
if not retrieved_webpages:
  print("Not found any approriate webpages! Please record your query again.")
else:
  print("Top relevant webpages found:")
  print(retrieved_webpages)

Top relevant webpages found:
['https://en.wikipedia.org/wiki/Abraham_Lincoln']


In [13]:
try:
  first_webpage_url = retrieved_webpages[0]
  page_content = extract_p_content(first_webpage_url, length_content=3)
  print("Content extracted:", page_content, sep="\n")
except IndexError:
  print("No URL to use")

Content extracted:
Abraham Lincoln (/ˈlɪŋkən/ LINK-ən; February 12, 1809 – April 15, 1865) was an American lawyer and statesman who served as the 16th president of the United States from 1861 until his assassination in 1865. Lincoln led the nation through the American Civil War and succeeded in preserving the Union, abolishing slavery, bolstering the federal government, and modernizing the U.S. economy.
Lincoln was born into poverty in a log cabin in Kentucky and was raised on the frontier, primarily in Indiana. He was self-educated and became a lawyer, Whig Party leader, Illinois state legislator, and U.S. Congressman from Illinois. In 1849, he returned to his law practice but became vexed by the opening of additional lands to slavery as a result of the Kansas–Nebraska Act of 1854. He reentered politics in 1854, becoming a leader in the new Republican Party, and he reached a national audience in the 1858 Senate campaign debates against Stephen Douglas. Lincoln ran for President in 186

## Perform text-to-speech

In [14]:
text_to_speech(page_content, lang=google_lang)

# **Vietnamese Virtual Assistant**

## Read audio file and perform ASR

### Use trained model

In [15]:
spectrogram = np.expand_dims(encode_single_sample(audio_filename), axis=0)
prediction = model.predict(spectrogram)
model_transcription = decode_batch_predictions(prediction)
print("Audio transcription: ", model_transcription[0])

Audio transcription:  h dư ngg


### Use library

In [16]:
r = sr.Recognizer()
my_audio = sr.AudioFile(audio_filename)
with my_audio as source:
  audio = r.record(source)

your_speech = r.recognize_google(audio, language="vi-VN")
print("Audio transcription: ", your_speech)

Audio transcription:  Sơn Tùng


## Perform google search and get the first wikipedia webpage

In [17]:
google_lang = "vi"

query = your_speech
retrieved_webpages = google_search(query, lang=google_lang)
if not retrieved_webpages:
  print("Not found any approriate webpages! Please record your query again.")
else:
  print("Top relevant webpages found:")
  print(retrieved_webpages)

Top relevant webpages found:
['https://vi.wikipedia.org/wiki/S%C6%A1n_T%C3%B9ng_M-TP']


In [18]:
try:
  first_webpage_url = retrieved_webpages[0]
  page_content = extract_p_content(first_webpage_url, length_content=3)
  print("Content extracted:", page_content, sep="\n")
except IndexError:
  print("No URL to use")

Content extracted:

Nguyễn Thanh Tùng (sinh ngày 5 tháng 7 năm 1994), thường được biết đến với nghệ danh Sơn Tùng M-TP, là một nam ca sĩ kiêm sáng tác nhạc, rapper và diễn viên người Việt Nam.



## Perform text-to-speech

In [19]:
text_to_speech(page_content, lang=google_lang)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
